In [0]:
display(dbutils.fs.ls("/mnt/Gen2/Orders/parquetFiles/"))

In [0]:
from pyspark.sql.functions import count

ordersDF = spark.read.format("parquet").load("/mnt/Gen2/Orders/parquetFiles/").withColumn('O_OrderYear',f.year("O_ORDERDATE"))

In [0]:
ordersDF.show()

In [0]:
import pyspark.sql.functions as f
display(ordersDF.filter("O_ORDERSTATUS = 'O'").groupBy("O_OrderYear","O_ORDERPRIORITY").agg(count("*").alias("TotalOrders")).orderBy("TotalOrders", ascending=False).limit(20))

In [0]:
ordersDF.write.format("delta").mode("overwrite").partitionBy("O_OrderYear").save("/mnt/Gen2/Orders/OrdersDelta/")

In [0]:
display(dbutils.fs.ls("/mnt/Gen2/Orders/OrdersDelta/"))

In [0]:
display(spark.sql("DROP TABLE  IF EXISTS Orders"))

display(spark.sql("CREATE TABLE Orders USING DELTA LOCATION '/mnt/Gen2/Orders/OrdersDelta'"))
                  
display(spark.sql("OPTIMIZE Orders ZORDER BY (O_ORDERPRIORITY)"))

In [0]:
from pyspark.sql.functions import count
ordersDeltaDF = spark.read.format("Delta").load("/mnt/Gen2/Orders/OrdersDelta/")
display(ordersDeltaDF.filter("O_ORDERSTATUS = 'O'").groupBy("O_OrderYear","O_ORDERPRIORITY").agg(count("*").alias("TotalOrders")).orderBy("TotalOrders", ascending=False).limit(20))

In [0]:
%sql
VACUUM ordersDeltaDF  -- Based on the delta table name
VACUUM '/mnt/Gen2/Orders/OrdersDelta/' -- Based on the delta table files location